In [1]:
pip install torch torchvision transformers


Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import os

In [3]:
# Define Paths
DATA_DIR = r"C:\Users\golla\Desktop\BTECH\8THSEM"
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# Load Data
def load_data(subset):
    path = os.path.join(DATA_DIR, subset)
    dataset = ImageFolder(root=path, transform=transform)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    return loader, dataset.class_to_idx  # Return dataset and label mapping

train_loader, class_to_idx = load_data("train")
valid_loader, _ = load_data("valid")
test_loader, _ = load_data("test")


In [5]:
# Load Pretrained ResNet-50 Model
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
resnet50.to(DEVICE).eval()

C:\Users\golla\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\golla\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [6]:
# Load Pretrained BERT Model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert = BertModel.from_pretrained("bert-base-uncased")
bert.to(DEVICE).eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [7]:
# Define Fake News Classification Model
class FakeNewsDetector(nn.Module):
    def __init__(self, resnet_model, bert_model):
        super(FakeNewsDetector, self).__init__()
        self.resnet = resnet_model
        self.bert = bert_model
        self.fc = nn.Linear(2048 + 768, 2)  # Concatenating ResNet (2048) and BERT (768)

    def forward(self, images, text_inputs):
        with torch.no_grad():
            image_features = self.resnet(images).squeeze()
            text_features = self.bert(**text_inputs).last_hidden_state[:, 0, :]
        combined = torch.cat((image_features, text_features), dim=1)
        return self.fc(combined)

In [8]:
# Initialize Model
model = FakeNewsDetector(resnet50, bert).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [9]:
# Training Loop (Assuming Text is Available)
for epoch in range(3):
    model.train()
    for images, labels in train_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Simulated Text Inputs (Replace with actual text data if available)
        texts = ["Fake news example"] * len(labels)
        text_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images, text_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.4783238470554352
Epoch 2, Loss: 0.8856081962585449
Epoch 3, Loss: 0.6239332556724548


In [10]:
# Save Model
torch.save(model.state_dict(), "fake_news_model.pth")


In [11]:
# Function to evaluate accuracy on a given dataset
def evaluate(loader, dataset_name):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            texts = ["Fake news example"] * len(labels)  # Replace with actual text data if available
            text_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

            outputs = model(images, text_inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f"{dataset_name} Accuracy: {accuracy:.4f}")

# Evaluate accuracy after all epochs
evaluate(train_loader, "Train")
evaluate(valid_loader, "Validation")
evaluate(test_loader, "Test")


Train Accuracy: 0.8462
Validation Accuracy: 0.8103
Test Accuracy: 0.8519


In [13]:
def forward(self, images, text_inputs):
    image_features = self.resnet(images)
    image_features = image_features.view(image_features.size(0), -1)  # Ensure batch dimension
    
    text_features = self.bert(**text_inputs).last_hidden_state[:, 0, :]
    text_features = text_features.view(text_inputs.input_ids.size(0), -1)  # Ensure batch dimension
    
    combined = torch.cat((image_features, text_features), dim=1)
    return self.fc(combined)


In [44]:
import torch
from torchvision import transforms
from PIL import Image
import pytesseract  

# Define the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to Tesseract-OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Image path
image_path = r"C:\Users\golla\Desktop\real.jpg"

# Image transformation (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

def extract_text_from_image(image_path):
    """Extract text using OCR with preprocessing."""
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    extracted_text = pytesseract.image_to_string(image, config="--psm 6")  
    return extracted_text.strip()

def predict_image_with_text(image_path):
    """Predicts if an image with text is Fake or Real using the trained model."""
    model.eval()

    # Load & preprocess image
    image = Image.open(image_path).convert("RGB")  
    image = transform(image).unsqueeze(0).to(DEVICE)  

    # Extract text from the image
    extracted_text = extract_text_from_image(image_path)
    print(f"Extracted Text: {extracted_text}")

    # Convert extracted text to model input
    text_inputs = tokenizer([extracted_text], padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = model(image, text_inputs)  
        print(f"Model Output Shape: {outputs.shape}")  # Debugging

        # Apply Sigmoid activation
        probabilities = torch.sigmoid(outputs).squeeze()

        # Extract single value correctly
        if probabilities.numel() == 1:
            probabilities = probabilities.item()
        else:
            probabilities = probabilities.cpu().numpy()[0]  # Get first element

        print(f"Sigmoid Probability: {probabilities}")

        # Adjust threshold for classification
        threshold = 0.6  
        predicted_label = int(probabilities >= threshold)

    # Assign class labels
    label = "Real" if predicted_label == 0 else "Fake"
    print(f"Predicted label: {label}")

# Run prediction
predict_image_with_text(image_path)


Extracted Text: "They would allow
just pretty much
anybody to be a
police officer."

Alvin Davis

former lieutenant in charge of

Memphis police recruiting
✅ Corrected Image Features Shape: torch.Size([1, 2048])
✅ Corrected Text Features Shape: torch.Size([1, 768])
Model Output Shape: torch.Size([1, 2])
Sigmoid Probability: 0.5726269483566284
Predicted label: Real


In [46]:
import torch
from torchvision import transforms
from PIL import Image
import pytesseract  

# Define the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to Tesseract-OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Image path
image_path = r"C:\Users\golla\Desktop\fake.jpg"

# Image transformation (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

def extract_text_from_image(image_path):
    """Extract text using OCR with preprocessing."""
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    extracted_text = pytesseract.image_to_string(image, config="--psm 6")  
    return extracted_text.strip()

def predict_image_with_text(image_path):
    """Predicts if an image with text is Fake or Real using the trained model."""
    model.eval()

    # Load & preprocess image
    image = Image.open(image_path).convert("RGB")  
    image = transform(image).unsqueeze(0).to(DEVICE)  

    # Extract text from the image
    extracted_text = extract_text_from_image(image_path)
    print(f"Extracted Text: {extracted_text}")

    # Convert extracted text to model input
    text_inputs = tokenizer([extracted_text], padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = model(image, text_inputs)  
        print(f"Model Output Shape: {outputs.shape}")  # Debugging

        # Apply Sigmoid activation
        probabilities = torch.sigmoid(outputs).squeeze()

        # Extract single value correctly
        if probabilities.numel() == 1:
            probabilities = probabilities.item()
        else:
            probabilities = probabilities.cpu().numpy()[0]  # Get first element

        print(f"Sigmoid Probability: {probabilities}")

        # Adjust threshold for classification
        threshold = 0.6  
        predicted_label = int(probabilities >= threshold)

    # Assign class labels
    label = "Real" if predicted_label == 0 else "Fake"
    print(f"Predicted label: {label}")

# Run prediction
predict_image_with_text(image_path)


Extracted Text: Fireds
Draining
the Swamp
✅ Corrected Image Features Shape: torch.Size([1, 2048])
✅ Corrected Text Features Shape: torch.Size([1, 768])
Model Output Shape: torch.Size([1, 2])
Sigmoid Probability: 0.6128115057945251
Predicted label: Fake
